In [ ]:
# Run this cell once to import required packages
!pip install pyexcelerate
# from pyexcelerate import Workbook, Color, Style, Font, Fill, Format
import pyexcelerate
!pip install openpyxl
import openpyxl
!pip install xlsxwriter
import xlsxwriter.workbook
import random

# Initialize the test data

In [ ]:
ROWS = 1000
COLUMNS = 100
BOLD = 1
ITALIC = 2
UNDERLINE = 4
RED_BG = 8

# python syntax to initialize list (Rows) of lists (Columns) filled with default value of 1
global testData, formatData
testData = [[1] * COLUMNS] * ROWS 
formatData = [[1] * COLUMNS] * ROWS

for row in range(ROWS):
    for col in range(COLUMNS):
        formatData[row][col] = random.randint(1, (1 << 4) - 1)
        

# PyExcelerate tests

In [ ]:
%%timeit
# Populate worksheet cells with values from testData[ROWS][COLS]
wb = pyexcelerate.Workbook()
wb.new_sheet("testData", data=testData)
wb.save("PyExcelerateValues.xlsx")

In [ ]:
%%timeit
# Format worksheet cells with formatting from formatData[ROWS][COLS]
wb = pyexcelerate.Workbook()
ws = wb.new_sheet("formatting")
bold = pyexcelerate.Style(font=pyexcelerate.Font(bold=True))
italic = pyexcelerate.Style(font=pyexcelerate.Font(italic=True))
underline = pyexcelerate.Style(font=pyexcelerate.Font(underline=True))
red = pyexcelerate.Style(fill=pyexcelerate.Fill(background=pyexcelerate.Color(255, 0, 0, 0)))
for row in range(ROWS):
    for col in range(COLUMNS):
        ws.set_cell_value(row + 1, col + 1, 1)
        style = pyexcelerate.Style()
        if formatData[row][col] & BOLD:
            style.font.bold = True
        if formatData[row][col] & ITALIC:
            style.font.italic = True
        if formatData[row][col] & UNDERLINE:
            style.font.underline = True
        if formatData[row][col] & RED_BG:
            style.fill.background = pyexcelerate.Color(255, 0, 0)
        ws.set_cell_style(row + 1, col + 1, style)
wb.save("PyExcelerateFormats.xlsx")


# XLSXWriter tests

In [ ]:
%%timeit
# Populate worksheet cells with values from testData[ROWS][COLS]
wb = xlsxwriter.workbook.Workbook("XLSXWriterValues.xlsx", {"constant_memory": True})
ws = wb.add_worksheet()
for row in range(ROWS):
    for col in range(COLUMNS):
        ws.write_number(row, col, 1)
wb.close()

In [ ]:
%%timeit
# Format worksheet cells with formatting from formatData[ROWS][COLS]
wb = xlsxwriter.workbook.Workbook("XLSXWriterFormats.xlsx", {"constant_memory": True})
ws = wb.add_worksheet()
cell_formats = []
for i in range(16):
    cell_format = wb.add_format()
    if i & BOLD:
        cell_format.set_bold()
    if i & ITALIC:
        cell_format.set_italic()
    if i & UNDERLINE:
        cell_format.set_underline()
    if i & RED_BG:
        cell_format.set_bg_color("red")
    cell_formats.append(cell_format)

for row in range(ROWS):
    for col in range(COLUMNS):
        ws.write_number(row, col, 1, cell_formats[formatData[row][col]])
wb.close()

# OpenPyXL tests

In [ ]:
%%timeit
# Populate worksheet cells with values from testData[ROWS][COLS]
wb = openpyxl.workbook.Workbook(write_only=True)
ws = wb.create_sheet()
ws.title = "Test 1"
for row in testData:
    ws.append(row)
wb.save("OpenPyXLValues.xlsx")

In [ ]:
%%timeit
# Format worksheet cells with formatting from formatData[ROWS][COLS]
wb = openpyxl.workbook.Workbook()
ws = wb.create_sheet()
ws.title = "Test 1"
for col_idx in range(COLUMNS):
    col = openpyxl.utils.get_column_letter(col_idx + 1)
    for row in range(ROWS):
        ws["%s%s" % (col, row + 1)].value = 1
        if formatData[row][col_idx] & BOLD:
            ws["%s%s" % (col, row + 1)].font = openpyxl.styles.Font(bold=True)
        if formatData[row][col_idx] & ITALIC:
            ws["%s%s" % (col, row + 1)].font = openpyxl.styles.Font(italic=True)
        if formatData[row][col_idx] & UNDERLINE:
            ws["%s%s" % (col, row + 1)].font = openpyxl.styles.Font(
                underline="single"
            )
        if formatData[row][col_idx] & RED_BG:
            ws["%s%s" % (col, row + 1)].fill = openpyxl.styles.PatternFill(
                fill_type=openpyxl.styles.fills.FILL_SOLID,
                start_color=openpyxl.styles.Color(openpyxl.styles.colors.RED),
                end_color=openpyxl.styles.Color(openpyxl.styles.colors.RED),
            )
        ws["%s%s" % (col, row + 1)].value = 1
wb.save("OpenPyXLFormats.xlsx")